# Figure 5a - Eigenspectra for p() and log p()

Panel 1 - Metric (approx Hess) eigenspectrum for full model of p and log p at initial point <br>
Panel 2 - Metric (approx Hess) Eigenvectors for full model of p and log p at initial point
 * Maybe visualise as vectors with blue-red (neg-pos) rectengles in a column vectors -> Shows that not axis-aligned

In [ ]:
using JLD
outp = load("figure3.jld", "outp");
outp_log10 = load("figure4.jld", "outp_log10");

In [ ]:
# Panel 1 - Eigenspectra
writecsv("CSV/figure_5a_panel1_orig.csv", hcat(1:size(outp[3][1,1],1), outp[3][1,1]))
writecsv("CSV/figure_5a_panel1_log10.csv", hcat(1:size(outp_log10[3][1,1],1), outp_log10[3][1,1]))

In [ ]:
# Panel2 - Eigenvectors (column vectors next to eachother, smallest to largest eigenvalue)
writecsv("CSV/figure_5a_panel2_orig.csv", outp[4][1,1])
writecsv("CSV/figure_5a_panel2_log10.csv", outp_log10[4][1,1])

## Generate error bars for eigenvectors and eigenvalues

1. Compute the Hessian in log param space around best fit
2. Shrink the hyperellipsoid defined by the Hessian such that the cost function change is < epsilon (5e-4)
3. Compute the Hessian in log param space around evenly distributed points on the hyperellipsoid of point 2
4. Store the explored parameter vectors, the costs (squared error between best fit and current parameters), and the eigendecomposition of the Hessian
5. Compute the mean and the variance of those eigendecompositions to show that on average we recover the true eigenvalues and initial directions





In [1]:
# Load the BK functions, the x_grid(Ca, V) that we evaluate over and the initial condition phi0
include("../BK_functions/bk_setup_script.jl"); 

In [2]:
# Copy some necessary code over from MBAM
x_loc = hcat(x_grid...);
model_cur(phi::AbstractArray, x::AbstractArray) = BK_simulator(phi, x, model_id=10)
data_vec = model_cur(phi0, x_loc)[:];
f_cost = phi1 -> sum((model_cur(phi1, x_loc)[:].-data_vec).^2); # Current cost function (quadratic)
g_cost = phi1 -> log_deriv_wrapper(f_cost, phi1); # Define which parameters we're taking in log space ([default: all])
f_res = phi1 -> (model_cur(phi1, x_loc)[:].-data_vec) # Return residuals
g_res = phi1 -> log_deriv_wrapper(f_res, phi1, log_specific=log_specific);

In [193]:
# Determine the hyperellipsoid around our best fit:
# hess1 = ForwardDiff.hessian(f_cost, phi0); # Lin hessian
hess2 = ForwardDiff.hessian(g_cost, log(phi0)); #log hessian
#Dorig, Vorig = eig(hess1)
Dorig, Vorig = eig(hess2)

([4.897121349714593e-5,0.0005651914699625763,0.0026368036677131236,0.09819393451560834,0.1682826953069511,0.6689945418932426,10.964452519206608,56.17485874674698],
8x8 Array{Float64,2}:
  0.891384    0.195203    0.322317   …  -0.165208   0.023164  -0.164101
  0.190804   -0.442983   -0.462876      -0.14188    0.300396  -0.123699
  0.115024   -0.396258   -0.388598      -0.161905  -0.115004  -0.405225
  0.0130497   0.0398636   0.0606904      0.470006   0.709311  -0.343254
  0.0405922   0.084609   -0.171099      -0.607664   0.12044    0.197849
 -0.0266633  -0.418244    0.352159   …   0.137564  -0.481374  -0.415317
 -0.382238    0.284259    0.238312      -0.509904   0.163247  -0.599683
  0.0846322   0.586654   -0.563101       0.239324  -0.346413  -0.323244)

In [201]:
# Evaluate the jac*jac' (~Hessian) on all "corner" points of a 0.1 radius L1 ball in log parameter space
D = Array(Any, 2^14) # Store eigenvalues
V = Array(Any, 2^14) # Store smallest eigenvector (or all)
Phis = Array(Any, 2^14)
Costs = Array(Any, 2^14)

log_phi0 = log(phi0)

# Copy stuff from MBAM main function

counter = 0;
using Iterators
for i in subsets(collect(1:8)) # Choose which parameters to do the step in
    for j in subsets(collect(1:length(i))) # Choose which direction to do the step in
        displace=zeros(8)
        direction = ones(length(i))./length(i) # such that it has correct norm
        direction[j] = -1./length(j); # such that it has correct norm
        displace[i]=1e-4*direction
        #@show displace
        
        # Compute the Hessian around the displace initial point
        #jac = map(Real, ForwardDiff.jacobian(f_res, phi0.*displace));
        phi_cur = log(phi0) + diagm(1./sqrt(abs(Dorig)))*Vorig*displace;
        #@show displace
        #@show exp(phi_cur)./phi0
        hess = map(Real, ForwardDiff.hessian(g_cost, phi_cur)) # Pass the log param vector to g_cost
        counter +=1
        #D[counter], V[counter] = eig(jac'*jac)
        D[counter], V[counter] = eig(hess)
        #V[counter] = V[counter][:,sortperm(D[counter])]
        #D[counter] = sort(D[counter])
        
        Phis[counter] = exp(phi_cur)
        Costs[counter] = f_cost(exp(phi_cur))
    end
end

D = D[1:counter]
V = V[1:counter]
Costs = Costs[1:counter];
Phis = Phis[1:counter];

using JLD
#save("figure5_hessianstability.jld", "D", D, "V", V, "Costs", Costs, "Phis", Phis)

In [202]:
Dsorted = deepcopy(D);
Vsorted = deepcopy(V);
which_eigenvec = 1;
for i = 1:counter
    Dsorted[i] = D[i][sortperm(abs(D[i]))]
    Vsorted[i] = V[i][:,sortperm(abs(D[i]))[which_eigenvec]]
end

In [203]:
[Vorig[:,which_eigenvec] mean(hcat(Vsorted...),2)./norm(mean(hcat(Vsorted...),2)) std(hcat(Vsorted...),2)./norm(mean(hcat(Vsorted...),2))]

8x3 Array{Float64,2}:
  0.891384    0.285398   2.97039 
  0.190804    0.0602339  0.385349
  0.115024   -0.250843   0.758611
  0.0130497   0.069634   0.325415
  0.0405922   0.292983   0.790986
 -0.0266633  -0.3246     1.30606 
 -0.382238   -0.078835   0.574344
  0.0846322   0.806059   2.1141  

In [204]:
[Dorig mean(hcat(Dsorted...),2) std(hcat(Dsorted...),2)]


8x3 Array{Float64,2}:
  4.89712e-5   -0.00413685  0.0150806
  0.000565191   0.0122338   0.0317072
  0.0026368    -0.0215348   0.183421 
  0.0981939    -0.0637649   0.359737 
  0.168283      0.114012    0.52756  
  0.668995      0.911338    0.420342 
 10.9645       10.3623      1.18924  
 56.1749       55.4291      3.76354  

In [205]:
hcat(Phis...)

8x6561 Array{Float64,2}:
  2.2e-6   7.86355e-6   6.15498e-7   2.90781e-6  …   2.42639e-6   1.77823e-6
  0.42     0.455098     0.387609     0.348599        0.460952     0.447132  
  0.1026   0.104924     0.100327     0.0949803       0.104787     0.103964  
  0.58     0.580242     0.579759     0.580738        0.578098     0.578279  
  3.9e-5   3.90386e-5   3.89614e-5   3.90805e-5      3.91441e-5   3.9117e-5 
  6.16     6.15799      6.16201      6.12858     …   6.17629      6.17473   
 30.4     30.3649      30.4351      30.4261         30.3999      30.4081    
  2.0      2.00023      1.99977      2.00157         2.0003       2.00021   

In [206]:
[maximum(hcat(Costs...)) mean(hcat(Costs...)) std(hcat(Costs...))]


1x3 Array{Float64,2}:
 2.21308  0.13657  0.252239

In [113]:
hcat(D...)

8x6561 Array{Float64,2}:
  4.89712e-5   39.0577     60.9321     …  59.8139     37.831      56.3699    
  0.000565191  10.0643      5.65997       11.1112      9.40531    10.3507    
  0.0026368     1.62703     2.99443        0.780037    1.64028     0.644322  
  0.0981939     1.06788    -1.99713       -0.266534    1.17618     0.224367  
  0.168283     -1.07358    -1.12851       -0.0223254  -1.09873     0.10661   
  0.668995      0.183278   -0.0562551  …   0.010238    0.171628    0.00819779
 10.9645       -0.0206237   0.113887       0.0513518  -0.0217148  -0.016123  
 56.1749       -0.512496    0.84554        0.327036   -0.504891   -0.0752944 

In [ ]:
diagm(1./sqrt(Dorig))*Vorig

In [ ]:
#jac = ForwardDiff.jacobian(f_res, phi0);
#Dorig, Vorig = eig(jac'*jac)

[hcat(Costs...); hcat(D...)]
hcat(D...)
#Vorig

In [192]:
# Visualise the explored Phis

using PlotlyJS
all_phis = hcat(Phis...)
plot(scatter(;x=all_phis[2,:][:], y=all_phis[4,:][:], mode="markers"))

# Figure 5b - Identifiability of reduced model

Panel 1 - Conditioning of the Hessian (every iteration step) <br>
Panel 2 - Global parameter landscape (do a param sweep in 3D)

In [ ]:
# Panel 1 - Condition numbers for p and log p (initially and after every reduction iteration)

cond_orig = zeros(size(outp[1],1)+1)
cond_log10 = zeros(size(outp_log10[1],1)+1)

cond_orig[1] = outp[3][1,1][end]./outp[3][1,1][1]
for i1 = 2:length(cond_orig)
    cond_orig[i1] = outp[3][i1-1,2][end]./outp[3][i1-1,2][2]
end

cond_log10[1] = outp_log10[3][1,1][end]./outp_log10[3][1,1][1]
for i1 = 2:length(cond_log10)
    cond_log10[i1] = outp_log10[3][i1-1,2][end]./outp_log10[3][i1-1,2][2]
end


# First column - reduction step
# Second column - condition number of metric (J'*J)

writecsv("CSV/figure_5b_panel1_orig.csv",  hcat(0:size(outp[1],1), cond_orig))
writecsv("CSV/figure_5b_panel1_log10.csv", hcat(0:size(outp_log10[1],1), cond_log10))

In [ ]:
# Panel 2 - Global parameter landscape

# Hard to do and we cannot really compare to original one - not sure if worth doing?